In [8]:
import pandas as pd
import numpy as np
import os
import re

# ============================================================
# 🧭 Y14_경찰청 지구대·파출소 수치화 전처리 (법정동코드 기반 시·구·동 파생)
# - 파일 로드
# - 법정동코드로 시·구·동 파생
# - 구별 지구대/파출소 수 집계 및 저장
# - 서울특별시 데이터 별도 저장
# ============================================================

# ===== 0) 경로 =====
input_file  = "/Users/mac/Documents/SORA_Project/data/raw/Y14_경찰청_전국 지구대 파출소 주소 현황_20241231_TM.csv"
output_dir  = "/Users/mac/Documents/SORA_Project/data/preprocessing"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "Y14_전처리_경찰청_전국 지구대 파출소 주소 현황_20241231_TM.csv")

# ===== 1) 로드 =====
df = pd.read_csv(input_file, sep="\t", encoding="utf-8-sig")
df.columns = df.columns.str.strip()
df.info()

# ===== 2) 법정동코드로 시·구·동 파생 =====
df["법정동코드_10자리"] = (
    df["법정동코드"]
    .fillna(0)
    .astype(np.int64)
    .astype(str)
    .str.zfill(10)
)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2045 entries, 0 to 2044
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   연번      2045 non-null   object 
 1   시도청     2024 non-null   object 
 2   경찰서     2024 non-null   object 
 3   관서명     2024 non-null   object 
 4   구분      2024 non-null   object 
 5   전화번호    2024 non-null   object 
 6   주소      2024 non-null   object 
 7   입력주소    2024 non-null   object 
 8   X       2024 non-null   float64
 9   Y       2024 non-null   float64
 10  CLSS    2024 non-null   object 
 11  PNU     2022 non-null   float64
 12  주소구분    2024 non-null   object 
 13  표준신주소   2010 non-null   object 
 14  표준구주소   2024 non-null   object 
 15  우편번호    1960 non-null   float64
 16  행정동코드   1997 non-null   float64
 17  행정동명    1997 non-null   object 
 18  법정동코드   1990 non-null   float64
 19  법정동명    1997 non-null   object 
dtypes: float64(6), object(14)
memory usage: 319.7+ KB


In [9]:
# ===== 2) 법정동코드로 시·구·동 파생 =====
df["법정동코드_10자리"] = (
    df["법정동코드"]
    .fillna(0)
    .astype(np.int64)
    .astype(str)
    .str.zfill(10)
)

df["시도코드"] = df["법정동코드_10자리"].str[:2]
df["시군구코드"] = df["법정동코드_10자리"].str[2:5]
df["법정동_코드"] = df["법정동코드_10자리"].str[5:10]

# 시 매핑
sido_map = {"11": "서울특별시"}  # 필요시 다른 시도 추가 가능
df["시"] = df["시도코드"].map(sido_map)

# 구 매핑
gu_map = {
    '110': '종로구', '140': '중구', '170': '용산구', '200': '성동구', '215': '광진구',
    '230': '동대문구', '260': '중랑구', '290': '성북구', '305': '강북구', '320': '도봉구',
    '350': '노원구', '380': '은평구', '410': '서대문구', '440': '마포구', '470': '양천구',
    '500': '강서구', '530': '구로구', '545': '금천구', '560': '영등포구', '590': '동작구',
    '620': '관악구', '650': '서초구', '680': '강남구', '710': '송파구', '740': '강동구'
}
df["구"] = df["시군구코드"].map(gu_map)

# 동 매핑 (법정동명 그대로 사용)
if "법정동명" in df.columns:
    df["동"] = df["법정동명"]
else:
    df["동"] = np.nan

In [19]:
# ===== 서울특별시 데이터만 필터링 =====
df_seoul = df[df["시"] == "서울특별시"].copy()

# ===== 서울특별시 구·동 결측치 상태 확인 =====
total_rows_seoul = len(df_seoul)
missing_cols_seoul = df_seoul[["구", "동"]].isnull().sum()
missing_total_seoul = missing_cols_seoul.sum()


print("\n🏙️ [결측치 확인 - 서울특별시]")
print(f"📊 총 행 수: {total_rows_seoul:,}행")
print("🚫 결측치 수 (시·구·동 컬럼별):")
print(missing_cols_seoul)
print(f"🧾 전체 결측치 총합: {missing_total_seoul}개")

print("\n📌 [서울특별시 데이터 샘플 10건]")
print(df_seoul.head(10))


🏙️ [결측치 확인 - 서울특별시]
📊 총 행 수: 234행
🚫 결측치 수 (시·구·동 컬럼별):
구    0
동    0
dtype: int64
🧾 전체 결측치 총합: 0개

📌 [서울특별시 데이터 샘플 10건]
    연번  시도청   경찰서    관서명   구분          전화번호                              주소  \
0    1  서울청  서울중부     을지  지구대  02-2279-1908              서울특별시 중구 퇴계로49길 13   
1    2  서울청  서울중부     광희  지구대  02-2233-1444             서울특별시 중구  퇴계로 375-1   
2    3  서울청  서울중부     약수  지구대  02-2234-8112             서울특별시 중구  동호로 5길 15   
4    5  서울청  서울중부     장충  파출소  02-2274-9003               서울특별시 중구  동호로 261   
5    6  서울청  서울중부     충무  파출소  02-2278-7710               서울특별시 중구  퇴계로 178   
6    7  서울청  서울중부  을지로3가  파출소  02-2266-2404                서울특별시 중구 충무로56-1   
7    8  서울청  서울종로   종로2가  지구대  02-3701-4301             서울특별시 종로구  종로17길 4    
8    9  서울청  서울종로     관수  파출소  02-3701-4302  서울특별시 종로구 삼일대로15길 19 1층(임시파출소)   
9   10  서울청  서울종로    세검정  파출소  02-3701-4507            서울특별시 종로구  세검정로 226    
10  11  서울청  서울종로     평창  파출소  02-3701-4308             서울특별시 종로구  평창문화로 46   

         

In [14]:
# ===== 5) 구별 지구대/파출소 수 집계 =====
df_seoul['지역구별 지구대/파출소 수'] = df_seoul.groupby('구')['구'].transform('size')
df_gu_count = (
    df_seoul.groupby('구')
    .size()
    .reset_index(name='지역구별 지구대/파출소 수')
    .sort_values(by='지역구별 지구대/파출소 수', ascending=False)
)

gu_summary_text = ", ".join([f"{row['구']} {row['지역구별 지구대/파출소 수']}개" for _, row in df_gu_count.iterrows()])
print("\n✅ [4단계] 서울 구별 집계 완료")
print(f"📊 지역구별 지구대/파출소 수 요약: {gu_summary_text}")


✅ [4단계] 서울 구별 집계 완료
📊 지역구별 지구대/파출소 수 요약: 종로구 20개, 중구 14개, 강남구 14개, 강서구 11개, 서초구 11개, 송파구 11개, 광진구 10개, 동대문구 10개, 영등포구 10개, 성북구 10개, 강북구 9개, 성동구 9개, 은평구 8개, 강동구 8개, 노원구 8개, 구로구 8개, 관악구 8개, 중랑구 8개, 양천구 8개, 서대문구 8개, 도봉구 8개, 동작구 7개, 마포구 7개, 금천구 5개, 용산구 4개


In [15]:
# ===== 6) 서울특별시 데이터만 저장 =====
output_file_seoul = os.path.join(
    output_dir,
    "Y14_전처리_경찰청_서울특별시_지구대_파출소_주소_현황_20241231_TM.csv"
)
df_seoul.to_csv(output_file_seoul, index=False, encoding="utf-8-sig")
print(f"\n💾 [5단계] Y14_전처리_경찰청_서울특별시_지구대_파출소_주소_현황_20241231_TM 저장 완료: {len(df_seoul):,}행")
print(f"📁 파일 경로: {output_file_seoul}")


💾 [5단계] Y14_전처리_경찰청_서울특별시_지구대_파출소_주소_현황_20241231_TM 저장 완료: 234행
📁 파일 경로: /Users/mac/Documents/SORA_Project/data/preprocessing/Y14_전처리_경찰청_서울특별시_지구대_파출소_주소_현황_20241231_TM.csv
